In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
sc=SparkSession.builder.appName("ML").getOrCreate()
data = sc.read.csv("/FileStore/tables/wc2018.csv", sep=',',header='true', 
                      inferSchema='true')



In [0]:
data.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [0]:
#Shows the name and the height by adding 1 to the height
data.select("FIFA Popular Name",data["Height"]+1).show(10)

+------------------+------------+
| FIFA Popular Name|(Height + 1)|
+------------------+------------+
|TAGLIAFICO Nicolas|         170|
|    PAVON Cristian|         170|
|    LANZINI Manuel|         168|
|    SALVIO Eduardo|         168|
|      MESSI Lionel|         171|
|  ANSALDI Cristian|         182|
|      BIGLIA Lucas|         176|
|       BANEGA Ever|         176|
| MASCHERANO Javier|         175|
|      DYBALA Paulo|         178|
+------------------+------------+
only showing top 10 rows



In [0]:
#Shows the player name and simultaneously checks whether or not they have height >170

agg = data.select("FIFA Popular Name",data["Height"]>170)
agg.show(10)



+------------------+--------------+
| FIFA Popular Name|(Height > 170)|
+------------------+--------------+
|TAGLIAFICO Nicolas|         false|
|    PAVON Cristian|         false|
|    LANZINI Manuel|         false|
|    SALVIO Eduardo|         false|
|      MESSI Lionel|         false|
|  ANSALDI Cristian|          true|
|      BIGLIA Lucas|          true|
|       BANEGA Ever|          true|
| MASCHERANO Javier|          true|
|      DYBALA Paulo|          true|
+------------------+--------------+
only showing top 10 rows



In [0]:
#shows FIFA popular name and 0 or 1 depending height over 170

data.select("FIFA Popular Name",data["Height"]/170).show(10)

+------------------+------------------+
| FIFA Popular Name|    (Height / 170)|
+------------------+------------------+
|TAGLIAFICO Nicolas|0.9941176470588236|
|    PAVON Cristian|0.9941176470588236|
|    LANZINI Manuel|0.9823529411764705|
|    SALVIO Eduardo|0.9823529411764705|
|      MESSI Lionel|               1.0|
|  ANSALDI Cristian|1.0647058823529412|
|      BIGLIA Lucas|1.0294117647058822|
|       BANEGA Ever|1.0294117647058822|
| MASCHERANO Javier|1.0235294117647058|
|      DYBALA Paulo|1.0411764705882354|
+------------------+------------------+
only showing top 10 rows



In [0]:
#name of shortest player
data.sort("Height").select("FIFA Popular Name").show(1)

+-----------------+
|FIFA Popular Name|
+-----------------+
|   YAHIA ALSHEHRI|
+-----------------+
only showing top 1 row



In [0]:
#who is tallest of all, print all attributes

data.sort("Height",ascending=False).show(1)

+-------+---+----+-----------------+----------+----------+--------------+------+------+
|   Team|  #|Pos.|FIFA Popular Name|Birth Date|Shirt Name|          Club|Height|Weight|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
|Croatia| 12|  GK|    KALINIC Lovre|03.04.1990|L. KALINIĆ|KAA Gent (BEL)|   201|    96|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
only showing top 1 row



In [0]:
#average height of argentine players

arg=data[data['Team']=="Argentina"]
arg.agg({'Height': 'mean'}).show()

+------------------+
|       avg(Height)|
+------------------+
|178.43478260869566|
+------------------+



In [0]:
ratings = sc.read.csv("/FileStore/tables/ratings.csv", sep=',',header='true', 
                      inferSchema='true')
movies = sc.read.csv("/FileStore/tables/movies.csv", sep=',',header='true', 
                      inferSchema='true')

In [0]:
ratings.show(4)




+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
+------+-------+------+----------+
only showing top 4 rows



In [0]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
movies.describe()

Out[431]: DataFrame[summary: string, movieId: string, title: string, genres: string]

In [0]:


#extract year
movies.createOrReplaceTempView('movies')
#a = sc.sql("""select substring(title,-5,4) as year from movies limit 5""")
a = sc.sql("select movieId,title,genres,CAST(substring(right(title,5),1, 4) as Integer) as year from movies where title like '%(%)'")
a.show()
a.dropna()
a.createOrReplaceTempView('movies')


+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
|     11|American Presiden...|Comedy|Drama|Romance|1995|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|
|     13|        Balto (1995)|Adventure|Animati...|1995|
|     14|        Nixon (1995)|               Drama|1995|
|     15|Cutthroat Island ...|A

In [0]:
#Find the list of oldest released movies
sc.sql("select year, title from movies order by year").show()

+----+--------------------+
|year|               title|
+----+--------------------+
|null|Big Bang Theory, ...|
|1902|Trip to the Moon,...|
|1915|Birth of a Nation...|
|1916|Intolerance: Love...|
|1916|20,000 Leagues Un...|
|1917|Immigrant, The (1...|
|1918|Dog's Life, A (1918)|
|1919|Billy Blazes, Esq...|
|1920|Cabinet of Dr. Ca...|
|1920|Golem, The (Golem...|
|1920|Mark of Zorro, Th...|
|1921|Play House, The (...|
|1921|Haunted House, Th...|
|1921|     Kid, The (1921)|
|1921|    Goat, The (1921)|
|1921|    Boat, The (1921)|
|1922|Dr. Mabuse: The G...|
|1922|Nosferatu (Nosfer...|
|1922|Paleface, The (1922)|
|1922|Haxan: Witchcraft...|
+----+--------------------+
only showing top 20 rows



In [0]:
#How many movies are released each year
sc.sql("select year,count(movieId) from movies group by year order by year").show()

+----+--------------+
|year|count(movieId)|
+----+--------------+
|null|             1|
|1902|             1|
|1915|             1|
|1916|             2|
|1917|             1|
|1918|             1|
|1919|             1|
|1920|             3|
|1921|             5|
|1922|             6|
|1923|             3|
|1924|             5|
|1925|             8|
|1926|             6|
|1927|             8|
|1928|             5|
|1929|             7|
|1930|             5|
|1931|            14|
|1932|            14|
+----+--------------+
only showing top 20 rows



In [0]:
ratings.createOrReplaceTempView('ratings')

In [0]:
#How many movies are there for each rating?

sc.sql("select rating,count(distinct movieId) from ratings group by rating order by rating").show()

+------+-----------------------+
|rating|count(DISTINCT movieId)|
+------+-----------------------+
|   0.5|                    868|
|   1.0|                   1959|
|   1.5|                   1204|
|   2.0|                   3130|
|   2.5|                   2409|
|   3.0|                   4771|
|   3.5|                   3612|
|   4.0|                   5141|
|   4.5|                   2454|
|   5.0|                   3127|
+------+-----------------------+



In [0]:
#How many users have rated each movie?
sc.sql("select movieId,count(userId) from ratings group by movieId").show()

+-------+-------------+
|movieId|count(userId)|
+-------+-------------+
|   1580|          190|
|   2659|            3|
|   3794|            5|
|   3175|           65|
|    471|           49|
|   1088|           53|
|   1342|           17|
|   1645|           60|
|   2366|           23|
|   6620|           17|
|   8638|           17|
|  96488|            4|
| 160563|            2|
|   7982|            3|
|   1238|           17|
|   1959|           30|
|    463|            7|
|   2122|           11|
|   1591|           15|
|   5518|            1|
+-------+-------------+
only showing top 20 rows



In [0]:
#what is the total rating for each movie
sc.sql("select movieId, sum(rating) from ratings group by movieId").show()

+-------+-----------+
|movieId|sum(rating)|
+-------+-----------+
|   1580|      696.0|
|   2659|       12.0|
|   3794|       17.0|
|   3175|      228.0|
|    471|      190.0|
|   1088|      178.0|
|   1342|       52.0|
|   1645|      207.5|
|   2366|       90.5|
|   6620|       62.0|
|   8638|       66.0|
|  96488|       15.0|
| 160563|        5.0|
|   7982|        9.5|
|   1238|       70.5|
|   1959|      114.0|
|    463|       24.0|
|   2122|       25.5|
|   1591|       40.5|
|   5518|        4.5|
+-------+-----------+
only showing top 20 rows



In [0]:
#what is the average rating for each movie
sc.sql("select movieId, avg(rating) from ratings group by movieId").show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|   1580| 3.663157894736842|
|   2659|               4.0|
|   3794|               3.4|
|   3175|3.5076923076923077|
|    471| 3.877551020408163|
|   1088| 3.358490566037736|
|   1342|3.0588235294117645|
|   1645|3.4583333333333335|
|   2366|3.9347826086956523|
|   6620|3.6470588235294117|
|   8638|3.8823529411764706|
|  96488|              3.75|
| 160563|               2.5|
|   7982|3.1666666666666665|
|   1238| 4.147058823529412|
|   1959|               3.8|
|    463|3.4285714285714284|
|   2122|2.3181818181818183|
|   1591|               2.7|
|   5518|               4.5|
+-------+------------------+
only showing top 20 rows

